# Import libraries

In [1]:
%matplotlib inline

import imageio
import pandas as pd
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
import random

import torch
import torchvision

import tensorflow as tf

from tqdm.notebook import tqdm
from os import path
from collections import namedtuple
from math import sqrt
from IPython.display import display
from PIL import Image, ImageColor, ImageDraw

from sklearn.cluster import KMeans

random.seed(42)

base_path = "D:\Vezdecode\CV\CV40"

# Determining the color of the car

In [2]:
def merge_channels(input_dir:str, output_dir:str):
    """
    Docstring:
        input_dir — name of the directory containing the dataset
        output_dir — name of the directory containing the RGB images
    """
    
    files = os.listdir(input_dir) # Reading files from the input directory
    indexes = [ind.split("_")[0] for ind in files] # Selection image indexes
    
    dict_with_channels = {} # Dictionary for storing channels and easy access to them
    merged_img = None # Image template for combining channels
    for index in tqdm(indexes):
        # Reading and filling in the dictionary for one picture with the appropriate channels
        for num, color in enumerate(["b", "g", "r"]): # The specific order of OpenCV colors
            path_img = path.join(input_dir, index+f"_{color}.jpg")
            
            merged_img = cv.imread(path_img)
            
            dict_with_channels[num] = merged_img[:, :, num]
        
        for num_channel in range(0, 3):
            merged_img[:, :, num_channel] = dict_with_channels[num_channel]
            
        cv.imwrite(path.join(output_dir, index+".jpg"), merged_img) # Saving the final image
         
def centroid_histogram(clt):
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins = numLabels)

    hist = hist.astype("float")
    hist /= hist.sum()

    return hist

def dominant_colors(perc_colors, centroids):
    if sum(centroids[0]) < 20:
        return centroids[1]
    else:
        return centroids[0]

def calc_metric(image, x=None, y=None, w=None, h=None):    
    if None not in [x, y, w, h]:
        image = image[y:y+h, x:x+w, :]
    
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    clt = KMeans(n_clusters=3)

    clt.fit(image)
        
    perc_colors = centroid_histogram(clt)
    max_color = dominant_colors(perc_colors, clt.cluster_centers_)
    
    return tuple((np.array(max_color)*255).round().astype("int"))

def find_car(image, model):
    results = model(image).crop(save=False)
    
    square = 0
    ind_car = 0
    cars = []
    
    for c in results:
        if "car" in c["label"]:
            cars.append(c)
    
    for num, car in enumerate(cars):
        if len(cars) == 1:
            break
        
        x = int(car["box"][0].cpu().item())
        y = int(car["box"][1].cpu().item())
        w = int(car["box"][2].cpu().item())
        h = int(car["box"][3].cpu().item())
        
        s = (w-x) * (h-y)

        if s > square:
            square = s
            ind_car = num
            
    if len(cars) == 0:
        return np.array([s.cpu().item() for s in results[0]["box"]]).round().astype("int")
    
    return np.array([s.cpu().item() for s in cars[ind_car]["box"]]).round().astype("int")

def find_color(input_dir, output_file="output_color.csv"):
    model = torch.hub.load("ultralytics/yolov5", "yolov5s")
    
    files = [path.join(input_dir, img) for img in os.listdir(input_dir)]
    
    result_df = pd.DataFrame(columns=["image", "color"])
    name_images = [] # A list for saving image names
    pred_colors = [] # A list for saving the prediction of the presence of a car in the image
    
    for img in tqdm(files):
        name_images.append(img.split("\\")[-1])
        
        image = cv.imread(img)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        x, y, w, h = find_car(image, model)
        
        color = calc_metric(image / 255., x, y, w, h)
        min_dist = 10000
        near_color = None
        
        for clr in dict_colors.keys():
            dist = ColorDistance(dict_colors[clr], color)
            
            if min_dist > dist:
                min_dist = dist
                near_color = clr
                
        pred_colors.append(near_color)
    
    result_df["image"] = name_images
    result_df["color"] = pred_colors

    result_df.to_csv(output_file, index=False, header=False)

    return result_df

def ColorDistance(rgb1, rgb2):
    rgb1 = np.array(rgb1)
    rgb2 = np.array(rgb2)
    
    d = ((rgb2[0] - rgb1[0]) ** 2 + (rgb2[1] - rgb1[1]) ** 2 + (rgb2[2] - rgb1[2]) ** 2) ** 0.5
    
    return d

In [3]:
channels_data = path.join(base_path, "resources\data")
merged_data_path = path.join(base_path, "resources\merged_data")

dir_with_data = merged_data_path

files = [path.join(base_path, dir_with_data, img) for img in os.listdir(dir_with_data)]

merge_channels(channels_data, merged_data_path)

  0%|          | 0/288 [00:00<?, ?it/s]

In [4]:
df = pd.read_csv(path.join(base_path, "resources\colors.csv"), header=None, names=["images", "color"])

In [5]:
dict_colors = {}

for color in ["silver", "black", "cyan", "yellow", "red", "green"]:
    if "cyan" in color:
        _key = "blue_cyan"
        dict_colors[_key] = ImageColor.getcolor(color, "RGB")
    elif "silver" in color:
        _key = "white_silver"
        dict_colors[_key] = ImageColor.getcolor(color, "RGB")
    else:
        dict_colors[color] = ImageColor.getcolor(color, "RGB")

dict_colors

{'white_silver': (192, 192, 192),
 'black': (0, 0, 0),
 'blue_cyan': (0, 255, 255),
 'yellow': (255, 255, 0),
 'red': (255, 0, 0),
 'green': (0, 128, 0)}

In [6]:
input_dir = path.join(base_path, "resources\merged_data")

output_df = find_color(input_dir)

Using cache found in C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-6-18 Python-3.8.11 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


  0%|          | 0/96 [00:00<?, ?it/s]

In [8]:
valid_target = pd.read_csv(
    path.join(base_path, "resources\\colors.csv"), 
    header=None, names=["image", "color"]
)

valid_target.head()

,image,color
0,00001.jpg,yellow
1,00002.jpg,yellow
2,00003.jpg,green
3,00004.jpg,black
4,00005.jpg,white_silver


In [13]:
K = sum(output_df["color"] == valid_target["color"])
N = len(valid_target)
score = max(0, K - 0.2 * N) * 40 / (N - 0.2 * N)
score

1.458333333333332